### 학습된 사이킷런 추정기 저장

<br>
<b>pickle 모듈</b>

In [4]:
import pickle
import os

In [5]:
#movieclassifier 디렉터리 안에 pkl_objects 서브디렉터리 만들어 직렬화된 파이썬 객체 저장
dest = os.path.join('movieclassifier','pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

In [6]:
import nltk
nltk.download('stopwords')#179개 불용어
from nltk.corpus import stopwords#불용어
stop = stopwords.words('english')
pickle.dump(stop,
           open(os.path.join(dest, 'stopwords.pkl'),'wb'),#wb매개변수 활용하여 이진모드로 파일열기
           protocol = 4)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#로지스틱회귀 모델 피팅하고 저장해야해서 이전 단원 코드 복붙
import pandas as pd
#그냥 깃헙에서 가져옴
df = pd.read_csv('C:/Users/LG/Desktop/2021/여름방학/MachineLearningTextbook/movie_data.csv',encoding='utf-8')
df.head()


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [9]:
import numpy as np
import re#정규표현식(regular expression)
def get_minibatch(doc_stream,size):#문서를 읽어 지정한 만큼 문서를 반환
    docs, y = [],[]
    try:
        for _ in range(size):#지정한 size만큼
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y
def stream_docs(path):#문서 하나씩 읽어서 반환되는 제너레이터 함수
    with open(path,'r',encoding = 'utf-8') as csv:
        next(csv)#헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text,label
def tokenizer(text):
    #텍스트 정제->불용어 제거-> 토큰으로 분리
    text = re.sub('<[^>]*>','',text)#정규표현식 사용
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub('[\W]+',' ',text.lower()) +#단어가 아닌 문자 모두 제거
            ''.join(emoticons).replace('-',''))#소문자로 바꿈, 이모티콘 변수를 처리 완료된 문자열 끝에 추가
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized
def preprocessor(text):#텍스트 데이터 정제
    text = re.sub('<[^>]*>','',text)#정규표현식 사용
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub('[\W]+',' ',text.lower()) +#단어가 아닌 문자 모두 제거
            ''.join(emoticons).replace('-',''))#소문자로 바꿈, 이모티콘 변수를 처리 완료된 문자열 끝에 추가
    return text

df['review'] = df['review'].apply(preprocessor)

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:,'review'].values
y_test=df.loc[25000:,'sentiment'].values
from sklearn.feature_extraction.text import HashingVectorizer#데이터 종류에 상관없이 사용 가능
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error = 'ignore',
                        n_features = 2**21,
                        preprocessor = None,
                        tokenizer=tokenizer)#해싱트릭사용
clf = SGDClassifier(loss = 'log',random_state = 1, max_iter =1)#로지스틱 회귀 모델(loss = 'log')
doc_stream = stream_docs(path = 'C:/Users/LG/Desktop/2021/여름방학/MachineLearningTextbook/movie_data.csv')

import pyprind
pbar = pyprind.ProgBar(45)#진행 막대
classes = np.array([0,1])
for _ in range(45):#45개 미니 배치
    X_train, y_train = get_minibatch(doc_stream, size = 1000)#1000개 문서로 구성된 각 미니 배치
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train,y_train,classes = classes)
    pbar.update()
    
X_test,y_test = get_minibatch(doc_stream,size=5000)#마지막 5000개 문서->평가
X_test = vect.transform(X_test)
clf = clf.partial_fit(X_test,y_test)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:37


In [10]:
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'),'wb'),#classifier.pkl이라고 로지스틱 회귀 모델 저장
           protocol=4)#로지스틱 회귀 직렬화하여 저장
           

#### Hashing Vectorizer 객체로 임포트할 수 있도록 파이썬 스크립트 작성(vectorizer.py)

In [1]:
#vectorizer를 임포트하고 분류기 복원
import pickle
import re
import os
from vectorizer import vect#from vectorizer.pyy라고 하면 안 불려와짐

clf = pickle.load(open(
    os.path.join('pkl_objects','classifier.pkl'),'rb'
))

In [2]:
import os
cwd = os.getcwd()
cwd#현재 디렉토리 출력(디렉토리 옮김)

'C:\\Users\\LG\\Desktop\\2021\\여름방학\\MachineLearningTextbook\\movieclassifier'

In [3]:
import numpy as np

label = {0:'음성',1:'양성'}#반환된 정수 값을 텍스트 레이블로 매핑하기 위한 딕셔너리

example = ['I love this movie']
X = vect.transform(example)
print("예측: %s\n확률: %.2f%%"%
      (label[clf.predict(X)[0]],
      np.max(clf.predict_proba(X))))

예측: 양성
확률: 0.81%


### 데이터를 저장하기 위해 SQLite 데이터베이스 설정

movieclassifier 디렉터리 안에 개로운 SQLite 데이터베이스를 생성하고 두 개의 영화리뷰 저장

In [14]:
import sqlite3

In [15]:
conn = sqlite3.connect('reviews.sqlite')#SQLite 데이터베이스 파일에 연결
c= conn.cursor()#커서를 통해 데이터베이스 레코드 조작
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db'#review_db 테이블과
         '(review TEXT, sentiment INTEGER, date TEXT)')#컬럼 3개(review, sentiment, date) 생성


In [16]:
example1 = 'I love this movie'
c.execute("INSERT INTO review_db"
         "(review, sentiment, date) VALUES"
         "(?, ?, DATETIME('now'))", (example1,1))#DATETIME(now): 날짜와 시간 추가

In [17]:
example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db"
         "(review, sentiment, date) VALUES"
         "(?, ?, DATETIME('now'))",#?,?: 영화리뷰텍스트(example1,example2)와 클래스 레이블(1,0)로 이루어진 튜플의 원소
         (example2,0))
conn.commit()#데이타베이스 변경사항 저장
conn.close()#데이타베이스와 연결 닫음

In [18]:
conn = sqlite3.connect('reviews.sqlite')#다시 연결
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date"
         " BETWEEN '2017-01-01 00:00:00' AND DATETIME('now')")#2017년부터 오늘까지 테이블에 추가된 모든 row 추출
results = c.fetchall()#추출
conn.close()#데이터베이스와 연결 닫음

In [20]:
print(results)

[('I love this movie', 1, '2021-07-22 05:51:01'), ('I disliked this movie', 0, '2021-07-22 05:51:02')]


### 플라스트 웹 애플리케이션 개발

In [21]:
!pip install flask

In [ ]:
from flask import Flask, render_template